In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import scipy.stats as st
from datetime import datetime
import gmaps
import os

# # Incorporated citipy to determine city based on latitude and longitude
# from citipy import citipy

# # Import API key
# from api_keys import g_key
# gmaps.configure(api_key=g_key)

In [34]:
# Make a reference to the starter csv file path and create df
prisoner_df = pd.read_csv('../Resources/Texas_Prison_Data.csv') 
prisoner_df

,SID Number,TDCJ Number,Name,Current Facility,Gender,Race,Age,Projected Release,Maximum Sentence Date,Parole Eligibility Date,Case Number,County,Offense Code,TDCJ Offense,Sentence Date,Offense Date,Sentence (Years),Last Parole Decision,Next Parole Review Date,Parole Review Status
0,234378,2174505,"LITTLE,AVA JESSUP",Young,F,W,75,10/01/2020,11/15/2025,09/26/2018,068770,Grayson,35990003,POSS CS 4-200G WID METH,12/14/2017,09/13/2017,8.0,Approved on 07/01/2020,NaN,NOT IN REVIEW PROCESS
1,524524,758486,"HARPER,DAVID JOHN JR",Estelle,M,W,91,01/01/9999,01/01/9999,08/03/2031,95-765-C,McLennan,36010001,INDECENCY WITH A CHILD,08/13/1996,09/15/1995,Life,None,08/03/2031,NOT IN REVIEW PROCESS
2,703267,841623,"RODRIGUEZ,THEODORE",Beto,M,W,82,08/03/2043,08/03/2043,02/01/2021,97-396-C,McLennan,11990003,AGG SEX ASLT,08/05/1998,06/01/1996,45.0,None,02/01/2021,IN PAROLE REVIEW PROCESS
3,708520,256174,"ALONZO,BENITO",Hospital Galveston,M,H,85,01/01/9999,01/01/9999,11/25/1985,49995,Travis,49030000,INTRODUCE DEADLY WPN PENAL INSTI,02/27/1976,06/01/1974,Life,Denied on 12/20/2019,12/2020,IN PAROLE REVIEW PROCESS
4,766785,365547,"BROWN,NARRIES EARL",Jester IV,M,W,84,01/01/9999,01/01/9999,08/24/2003,F83-89728-HI,Dallas,11220000,AGG SEX ABUSE CHILD/U14,08/24/1983,03/15/1983,Life,Denied on 12/05/2016,Unavailable at this time.,NOT IN REVIEW PROCESS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120702,50824296,2192344,"PINEDA-NAVA,CRISTIAN",Clemens,M,H,22,01/20/2056,01/20/2056,01/20/2036,241-0316-16,Smith,11990004,AGG SEXUAL ASLT CHILD,02/22/2018,10/15/2015,40.0,None,01/20/2036,NOT IN REVIEW PROCESS
120703,50824299,2180043,"AARON,LYNDSEY NICOLE",Crain,F,W,22,09/18/2021,09/18/2021,NaN,D-1-DC-17-300046,Travis,13990063,ASLT PUB SERV,02/09/2018,01/03/2017,4.0,Denied on 03/09/2020,03/2021,NOT IN REVIEW PROCESS
120704,50824431,2173750,"WRIGHT,LARRY LEE",Robertson,M,W,73,01/15/2043,01/15/2043,07/16/2030,CR02467,Red River,11990004,AGG SEX ASLT CHILD,01/17/2018,01/15/2016,25.0,None,07/16/2030,NOT IN REVIEW PROCESS
120705,50824569,2272406,"COX,CARRIE BETH",San Saba,F,W,25,04/18/2024,04/18/2024,11/15/2019,CR-1090-19-D,Hidalgo,13150004,AGG ASLT SBI,07/10/2019,01/19/2019,5.0,Denied on 11/25/2019,11/2020,IN PAROLE REVIEW PROCESS


In [3]:
# prisoner_df.isna().sum()

In [4]:
# prisoner_df.dtypes

In [5]:
# This df is from the Texas Department of Criminal Justice (TDCJ)
# It has ALL the criminal code assignments - those that are currently being
# used AND those no longer used.  Some individuals have codes assigned that are no
# longer active.  This df has a column (Offense_Category)that can be used as a key 
# to link many different specific, detailed codes (Offense Codes) together.

offense_df = pd.read_csv('../Resources/offenseCodes_ut8.csv', encoding="utf-8")
# offense_df.head()

In [6]:
# offense_df.dtypes

In [7]:
# Create project specific df for merging. 
# This step creates a smaller dataframe that will be used to merge 
# with the prisoner df on the key offense cateogry

offproject_df = offense_df.loc[:,["Offense Code", "Offense_Category"]]
# offproject_df.head()

In [8]:
# This step actually creates the "bins" - the last column "Offense_Category" can be sorted on
# and capture the respective individuals by offense type.  The "ffill" will place a Nan in 
# the "Offense_Category" column if there is no value in that location.  
# The "merge_ordered" maintains the integrity of the elements in the order of the main df,
# prisoner_df, while matching the Offense_Cateogry" with the respective "Offense Code".
# The prisoner df has approximately 208K rows with many duplicated Offense Code.  
# The offproject df has 2,738 rows of 62 unique Offense Cateogries and related Offense Codes
# which results in the 2,738 rows on information in the df.  The Offense Code column is mapped
# to the respective value in the prisoner df and Offense_Category is assigned to the respective row.

binlookup_df = pd.merge_ordered(prisoner_df, offproject_df, fill_method='ffill', left_by='Offense Code')
binlookup_df.head()

,SID Number,TDCJ Number,Name,Current Facility,Gender,Race,Age,Projected Release,Maximum Sentence Date,Parole Eligibility Date,...,County,Offense Code,TDCJ Offense,Sentence Date,Offense Date,Sentence (Years),Last Parole Decision,Next Parole Review Date,Parole Review Status,Offense_Category
0,234378,2174505,"LITTLE,AVA JESSUP",Young,F,W,75,10/01/2020,11/15/2025,09/26/2018,...,Grayson,35990003,POSS CS 4-200G WID METH,12/14/2017,09/13/2017,8.0,Approved on 07/01/2020,NaN,NOT IN REVIEW PROCESS,2200
1,1116232,1366652,"SIDLE,JOHN ARTHUR",Pack,M,W,78,05/02/2021,06/18/2033,03/08/2016,...,Dallas,35990003,UNLAW POSS WIT C/S-METHAM,04/06/2006,09/03/2004,25.0,Approved on 09/17/2020,NaN,NOT IN REVIEW PROCESS,2200
2,1409034,729430,"LERMA,GILBERT LOPEZ",Wynne,M,H,70,01/01/9999,01/01/9999,08/22/2020,...,Williamson,35990003,DEL C/S,09/19/1995,09/28/1994,Life,Denied on 07/16/2020,07/2021,NOT IN REVIEW PROCESS,2200
3,1468825,2205051,"EVANS,DAYTON BUD",Pack,M,W,70,04/21/2028,03/11/2043,10/20/2020,...,Grayson,35990003,POSS CONT SUB PG1 4-200 WITD,03/12/2018,03/31/2017,25.0,Denied on 08/13/2020,08/2021,NOT IN REVIEW PROCESS,2200
4,1473183,2038325,"GREEN,ROGER DALE SR",Duncan,M,W,73,08/16/2025,06/27/2040,02/14/2018,...,Johnson,35990003,DEL CONT SUB 4G-200G,11/10/2015,05/05/2015,25.0,Denied on 02/06/2020,02/2021,NOT IN REVIEW PROCESS,2200


In [9]:
# binlookup_df.isna().sum()

In [10]:
#Created this to keep from corrupting the good file
#did not need to do this - did not want issues a 2:37am
backup_df = binlookup_df.copy()

In [11]:
#This identifies the unique Offense Catrgory in the prisoner (newly creaked binlookup)
# df.  There are a total of 62 offense catories overall and 45 were matched to the prisoners
# in the prisoner df when creating the merged_ordered df.

binlookup_df["Offense_Category"].unique()

array([2200,  400, 3200,  200,  500, 4200,  600, 1500, 3700, 1900,  900,
       1600, 1800, 3500, 2300,  300, 4400, 4600, 4500,  700, 3400, 3100,
       4900, 3600, 1200, 2400, 2900, 2700,  100, 4100, 6200, 2100, 2600,
       4300, 3300, 3900, 1100, 2500, 6100, 5300, 1400, 5200, 4700, 5500,
       5400], dtype=int64)

In [12]:
# Create bins in which to place values based upon Offense Categories
# The step serves the purpose of assigning a label to the respective offense category
# must have 1 extra bin then actual catorgies - hence the 0 at the beginning

bins = [0, 100, 200, 300, 400, 500, 600, 700, 900, 1100, 1200, 1400, 
        1500, 1600, 1800, 1900, 2100, 2200, 2300, 2400, 2500, 2600, 
        2700, 2900, 3100, 3200, 3300, 3400, 3500, 3600, 3700, 3900, 
        4100, 4200, 4300, 4400, 4500, 4600, 4700, 4900, 5200, 5300, 
        5400, 5500, 6100, 6200]

# Create labels for these bins
group_labels = ["Human_Trafficking", "Murder", "Kidnapping", "Sexual_Assault", "Robbery", "Assault",
                "Coercion_Politics", "Harrassment", "Terrorism", "Arson", "Unlawful_Use", "Burglary", "Theft",
                "Unlawful_Rep", "Fraud_Forgery", "Vandalism", "Drugs", "Sexual_Related_Materials", "Child_Endanger", "Bigomy",
                "Elderly_at_Risk_Adult_Endanger", "Violate_Order", "Prostitution", "Evading_Arrest", "Criminal_Escape",
                "Perjury_Contemp", "Jump_Bail_No_Show", "Interference_Records", "Bribery", "Weapon_Related", 
                "Riot_Obstruction", "Vehicle_Incident", "Alcohol_Incident", "Animal_Related", "Agriculture_Related",
                "Health_Code", "Hazardous_Material", "Discrimination_Civil_Rights", "Record_Tampering", "Tax_Evasion",
                "Pollution", "Money_Laundering", "Negligence", "Business_License", "Organized_Crime"]

In [13]:
# Slice the data and place it into bins
# pd.cut(binlookup_df["Offense_Category"], bins, labels=group_labels).head()

In [14]:
# Place the data series (offense category label) into a new (last) column inside of the DataFrame

binlookup_df["Bin_Cat"] = pd.cut(binlookup_df["Offense_Category"], bins, labels=group_labels)
binlookup_df.head()

,SID Number,TDCJ Number,Name,Current Facility,Gender,Race,Age,Projected Release,Maximum Sentence Date,Parole Eligibility Date,...,Offense Code,TDCJ Offense,Sentence Date,Offense Date,Sentence (Years),Last Parole Decision,Next Parole Review Date,Parole Review Status,Offense_Category,Bin_Cat
0,234378,2174505,"LITTLE,AVA JESSUP",Young,F,W,75,10/01/2020,11/15/2025,09/26/2018,...,35990003,POSS CS 4-200G WID METH,12/14/2017,09/13/2017,8.0,Approved on 07/01/2020,NaN,NOT IN REVIEW PROCESS,2200,Drugs
1,1116232,1366652,"SIDLE,JOHN ARTHUR",Pack,M,W,78,05/02/2021,06/18/2033,03/08/2016,...,35990003,UNLAW POSS WIT C/S-METHAM,04/06/2006,09/03/2004,25.0,Approved on 09/17/2020,NaN,NOT IN REVIEW PROCESS,2200,Drugs
2,1409034,729430,"LERMA,GILBERT LOPEZ",Wynne,M,H,70,01/01/9999,01/01/9999,08/22/2020,...,35990003,DEL C/S,09/19/1995,09/28/1994,Life,Denied on 07/16/2020,07/2021,NOT IN REVIEW PROCESS,2200,Drugs
3,1468825,2205051,"EVANS,DAYTON BUD",Pack,M,W,70,04/21/2028,03/11/2043,10/20/2020,...,35990003,POSS CONT SUB PG1 4-200 WITD,03/12/2018,03/31/2017,25.0,Denied on 08/13/2020,08/2021,NOT IN REVIEW PROCESS,2200,Drugs
4,1473183,2038325,"GREEN,ROGER DALE SR",Duncan,M,W,73,08/16/2025,06/27/2040,02/14/2018,...,35990003,DEL CONT SUB 4G-200G,11/10/2015,05/05/2015,25.0,Denied on 02/06/2020,02/2021,NOT IN REVIEW PROCESS,2200,Drugs


In [15]:
rcbinlookup_df = binlookup_df.rename(columns={"SID Number": "SID_Number", 
                            "TDCJ Number": "TDCJ_Number",
                            "Current Facility" : "Current_Facility",
                            "Projected Release" : "Projected_Release",
                            "Maximum Sentence Date" : "Max_Sentence_Date",
                            "Parole Eligibility Date" : "Parole_Eligible_Date",
                            "Case Number" : "Case_Number",
                            "Offense Code" :"Offense_Code",
                            "TDCJ Offense" :"TDCJ_Offence",
                            "Sentence Date" : "Sentence_Date",
                            "Offense Date": "Offense_Date",
                            "Sentence (Years)" : "Sentence_Years",
                            "Last Parole Decision" : "Last_Parole_Decision",
                            "Next Parole Review Date" : "Next_Parole_Review",
                            "Parole Review Status" : "Parole_Review_Status"})

In [16]:
# rcbinlookup_df["Sentence_Years"].unique()

In [17]:
# rcbinlookup_df.dtypes

In [18]:
# rcbinlookup_df["Offense_Code"].unique()

In [19]:
rcbinlookup_df=rcbinlookup_df.replace(to_replace="Life",value="50.9")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="LWOP", value="100.9")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="Capital Life", value = "101.9") 
rcbinlookup_df=rcbinlookup_df.replace(to_replace="Death", value = "150.9")

#Not able to do NaN with this method
# Index 71997 SID_Number 4340086   TDCJ_Number 2319589 is the NaN
# rcbinlookup_df=rcbinlookup_df.replace(to_replace="NaN", value = "3999.9")

rcbinlookup_df.head()

,SID_Number,TDCJ_Number,Name,Current_Facility,Gender,Race,Age,Projected_Release,Max_Sentence_Date,Parole_Eligible_Date,...,Offense_Code,TDCJ_Offence,Sentence_Date,Offense_Date,Sentence_Years,Last_Parole_Decision,Next_Parole_Review,Parole_Review_Status,Offense_Category,Bin_Cat
0,234378,2174505,"LITTLE,AVA JESSUP",Young,F,W,75,10/01/2020,11/15/2025,09/26/2018,...,35990003,POSS CS 4-200G WID METH,12/14/2017,09/13/2017,8.0,Approved on 07/01/2020,NaN,NOT IN REVIEW PROCESS,2200,Drugs
1,1116232,1366652,"SIDLE,JOHN ARTHUR",Pack,M,W,78,05/02/2021,06/18/2033,03/08/2016,...,35990003,UNLAW POSS WIT C/S-METHAM,04/06/2006,09/03/2004,25.0,Approved on 09/17/2020,NaN,NOT IN REVIEW PROCESS,2200,Drugs
2,1409034,729430,"LERMA,GILBERT LOPEZ",Wynne,M,H,70,01/01/9999,01/01/9999,08/22/2020,...,35990003,DEL C/S,09/19/1995,09/28/1994,50.9,Denied on 07/16/2020,07/2021,NOT IN REVIEW PROCESS,2200,Drugs
3,1468825,2205051,"EVANS,DAYTON BUD",Pack,M,W,70,04/21/2028,03/11/2043,10/20/2020,...,35990003,POSS CONT SUB PG1 4-200 WITD,03/12/2018,03/31/2017,25.0,Denied on 08/13/2020,08/2021,NOT IN REVIEW PROCESS,2200,Drugs
4,1473183,2038325,"GREEN,ROGER DALE SR",Duncan,M,W,73,08/16/2025,06/27/2040,02/14/2018,...,35990003,DEL CONT SUB 4G-200G,11/10/2015,05/05/2015,25.0,Denied on 02/06/2020,02/2021,NOT IN REVIEW PROCESS,2200,Drugs


In [20]:
# rcbinlookup_df["Sentence_Years"].unique()

In [21]:
#these are global changes - so need to check at some point for data in other columns being inadvertently changed
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2024.9",value="6.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2092.7", value="8.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2027.0", value = "10.0") 
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2026.4", value = "11.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2027.1",  value = "10.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2024.8",value="6.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2024.7", value="7.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2023.5", value = "5.0") 
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2026.3", value = "8.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2020.8",  value = "5.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2022.4",value="8.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2019.8", value="10.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2022.2", value = "5.0") 
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2024.0", value = "6.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2029.9",  value = "10.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2022.3",value="6.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2027.9", value="8.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2023.4", value = "6.0") 
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2021.4", value = "6.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2021.5",value="2.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2022.9", value="3.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2022.8", value = "4.0") 
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2023.0", value = "5.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2024.3",  value = "6.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2021.6",value="4.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2021.2", value="5.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2022.7", value = "5.0") 
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2023.9", value = "4.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2025.1",  value = "9.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2020.5",value="1.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2019.5", value="4.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2029.5", value = "10.0") 
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2021.9", value = "10.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2025.0",  value = "6.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2024.5",value="5.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2022.6", value="3.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2020.2", value = "2.0") 
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2030.0", value = "11.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2023.6",  value = "5.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2024.2",value="6.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2029.7",  value = "8.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2020.9",value="2.0")

# rcbinlookup_df

In [22]:
# rcbinlookup_df["Sentence_Years"].unique()

In [23]:
# rcbinlookup_df.loc[rcbinlookup_df["Sentence_Years"] == "800.0"]

In [24]:
# nanloc_df = rcbinlookup_df[pd.to_numeric(rcbinlookup_df['Sentence_Years'], errors='coerce').isnull()]
# nanloc_df

# 	SID_Number	TDCJ_Number	Name	Current_Facility	Gender	Race	Age	Projected_Release	Max_Sentence_Date	Parole_Eligible_Date	...	Offense_Code	TDCJ_Offence	Sentence_Date	Offense_Date	Sentence_Years	Last_Parole_Decision	Next_Parole_Review	Parole_Review_Status	Offense_Category	Bin_Cat
# 71997	4340086	2319589	DAVIS,CHARLES ELLIS	Estelle	M	W	49	06/29/2021	NaN	NaN	...	35990015	POSS CONT SUB 1-4G	04/14/2020	07/30/2019	NaN	NaN	NaN	NaN	2200	Drugs



In [25]:
rcbinlookup_df.loc[71997,"Sentence_Years"] = 10.0

In [26]:
# rcbinlookup_df["Sentence_Years"].unique()

In [27]:
# Create age bins
# must have 1 extra bin then actual catorgies - hence the 0 at the beginning

agebins = [0, 19.9, 29.9, 39.9, 49.9, 59.9, 69.9, 79.9, 89.9, 99.9, 199.9]

# Create labels for these bins
agegroup_labels = ["Less_Than_20", "20_to_30", "30_to_40",
                   "40_to_50", "50_to_60", "60_to_70",
                   "70_to_80", "80_to_90", "90_to_100", "Greater_than_100"]   

In [28]:
# Create new column - Age_Bin

rcbinlookup_df["Age_Bin"] = pd.cut(rcbinlookup_df["Age"], agebins, labels=agegroup_labels)
rcbinlookup_df.head()

,SID_Number,TDCJ_Number,Name,Current_Facility,Gender,Race,Age,Projected_Release,Max_Sentence_Date,Parole_Eligible_Date,...,TDCJ_Offence,Sentence_Date,Offense_Date,Sentence_Years,Last_Parole_Decision,Next_Parole_Review,Parole_Review_Status,Offense_Category,Bin_Cat,Age_Bin
0,234378,2174505,"LITTLE,AVA JESSUP",Young,F,W,75,10/01/2020,11/15/2025,09/26/2018,...,POSS CS 4-200G WID METH,12/14/2017,09/13/2017,8.0,Approved on 07/01/2020,NaN,NOT IN REVIEW PROCESS,2200,Drugs,70_to_80
1,1116232,1366652,"SIDLE,JOHN ARTHUR",Pack,M,W,78,05/02/2021,06/18/2033,03/08/2016,...,UNLAW POSS WIT C/S-METHAM,04/06/2006,09/03/2004,25.0,Approved on 09/17/2020,NaN,NOT IN REVIEW PROCESS,2200,Drugs,70_to_80
2,1409034,729430,"LERMA,GILBERT LOPEZ",Wynne,M,H,70,01/01/9999,01/01/9999,08/22/2020,...,DEL C/S,09/19/1995,09/28/1994,50.9,Denied on 07/16/2020,07/2021,NOT IN REVIEW PROCESS,2200,Drugs,70_to_80
3,1468825,2205051,"EVANS,DAYTON BUD",Pack,M,W,70,04/21/2028,03/11/2043,10/20/2020,...,POSS CONT SUB PG1 4-200 WITD,03/12/2018,03/31/2017,25.0,Denied on 08/13/2020,08/2021,NOT IN REVIEW PROCESS,2200,Drugs,70_to_80
4,1473183,2038325,"GREEN,ROGER DALE SR",Duncan,M,W,73,08/16/2025,06/27/2040,02/14/2018,...,DEL CONT SUB 4G-200G,11/10/2015,05/05/2015,25.0,Denied on 02/06/2020,02/2021,NOT IN REVIEW PROCESS,2200,Drugs,70_to_80


In [29]:
## Cleanup Data Column for Counties 

rc_un = rcbinlookup_df['County'].unique()
rc_un = np.sort(rc_un)
rc_un = pd.DataFrame(rc_un)
rc_un.to_csv('../Resources/rc_counties.csv', index=False)


In [30]:
# Replace Lasalle with La Salle

rcbinlookup_df = rcbinlookup_df.replace(to_replace="Lasalle", value= "La Salle")

In [31]:
# Loading and manipulating population data for 2020 pop data 

countyall_df = pd.read_csv('../Resources/pop_data_with_projection.csv')
texas2020_df = countyall_df.query('year == 2020')
county2020_df = texas2020_df.loc[texas2020_df['age_group'] == "ALL"]
notex_df = county2020_df.drop([60])
proj_county2020 = notex_df.loc[:,["area_name", "total"]]
rcproj_county2020_df = proj_county2020.rename(columns={"area_name": "County", "total" : "Population_2020"})

pd.set_option('display.max_columns', None)
rcproj_county2020_df.head(65)


,County,Population_2020
306,Anderson,58199
552,Andrews,22269
798,Angelina,90437
1044,Aransas,27699
1290,Archer,8344
...,...,...
15066,Denton,897953
15312,Dewitt,21737
15558,Dickens,2174
15804,Dimmit,11743


In [32]:
# Merge Prison and Population database
mother_ship = pd.merge_ordered(rcbinlookup_df, rcproj_county2020_df, fill_method='ffill', left_by='County')
mother_ship.isna().sum()
values = {"Population_2020": 21737}
mother_ship.fillna(value=values)
mother_ship.isna().sum()
#county_error = mother_ship[mother_ship['Population_2020'].isnull()]
#county_error.head()
#county_error = county_error['County'].unique()

SID_Number                  0
TDCJ_Number                 0
Name                        0
Current_Facility            0
Gender                      0
Race                        0
Age                         0
Projected_Release           0
Max_Sentence_Date          28
Parole_Eligible_Date     9017
Case_Number                99
County                      0
Offense_Code                0
TDCJ_Offence                0
Sentence_Date            1233
Offense_Date                0
Sentence_Years              0
Last_Parole_Decision     7815
Next_Parole_Review      22397
Parole_Review_Status     7815
Offense_Category            0
Bin_Cat                     0
Age_Bin                     0
Population_2020             0
dtype: int64

In [33]:
mother_ship

,SID_Number,TDCJ_Number,Name,Current_Facility,Gender,Race,Age,Projected_Release,Max_Sentence_Date,Parole_Eligible_Date,Case_Number,County,Offense_Code,TDCJ_Offence,Sentence_Date,Offense_Date,Sentence_Years,Last_Parole_Decision,Next_Parole_Review,Parole_Review_Status,Offense_Category,Bin_Cat,Age_Bin,Population_2020
0,234378,2174505,"LITTLE,AVA JESSUP",Young,F,W,75,10/01/2020,11/15/2025,09/26/2018,068770,Grayson,35990003,POSS CS 4-200G WID METH,12/14/2017,09/13/2017,8.0,Approved on 07/01/2020,NaN,NOT IN REVIEW PROCESS,2200,Drugs,70_to_80,131710
1,1468825,2205051,"EVANS,DAYTON BUD",Pack,M,W,70,04/21/2028,03/11/2043,10/20/2020,068250,Grayson,35990003,POSS CONT SUB PG1 4-200 WITD,03/12/2018,03/31/2017,25.0,Denied on 08/13/2020,08/2021,NOT IN REVIEW PROCESS,2200,Drugs,70_to_80,131710
2,1505838,2119905,"PENNINGTON,RICKEY DWAYNE",Pack,M,W,68,12/21/2028,12/21/2028,04/11/2018,067845,Grayson,35990003,POSS CS WITD-METH 4-200G,02/08/2017,12/21/2016,12.0,Denied on 01/02/2020,12/2020,IN PAROLE REVIEW PROCESS,2200,Drugs,60_to_70,131710
3,2141308,2153019,"JACOBS,ANTHONY",Ellis,M,W,64,12/18/2023,06/14/2033,02/28/2019,068249,Grayson,35990003,POSS CS 4-200G WID METH,07/27/2017,10/11/2016,16.0,Denied on 02/19/2020,02/2021,NOT IN REVIEW PROCESS,2200,Drugs,60_to_70,131710
4,2360914,2290868,"WALDEN,LONNIE WAYNE",Gurney,M,W,62,04/02/2033,05/28/2049,NaN,071105,Grayson,35990003,MAN DEL CS PG1 4-200G METH,10/24/2019,05/29/2019,30.0,None,Unavailable at this time.,NOT IN REVIEW PROCESS,2200,Drugs,60_to_70,131710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120702,5826119,2251746,"MUNOZ,JOSE JR",Polunsky,M,H,40,04/26/2022,07/24/2026,08/19/2019,M-19-0011-CR-B,McMullen,48990012,TAMPERING W/A WITNESS,02/04/2019,07/24/2018,8.0,Denied on 07/07/2020,07/2021,NOT IN REVIEW PROCESS,3100,Evading_Arrest,40_to_50,783
120703,4231127,2043573,"BRAVO,CARLOS RUIZ",Wynne,M,H,61,06/30/2033,04/25/2060,12/31/2019,M-13-0007-CR-B,McMullen,35620013,POSS MARIJUANA M/2000LBS,12/02/2015,10/19/2012,45.0,Denied on 12/09/2019,12/2020,IN PAROLE REVIEW PROCESS,2200,Drugs,60_to_70,783
120704,7987519,2284322,"MORRIS,JUSTIN LEE",Dominguez,M,W,31,05/30/2021,05/30/2021,NaN,M-19-0010-CR-B,McMullen,23990008,(LIO) THEFT OF FIREARM,07/08/2019,07/21/2017,2.0,NaN,NaN,NaN,1600,Theft,30_to_40,783
120705,7273574,2311921,"GONZALEZ,MIGUEL",Willacy County,M,H,30,07/12/2022,02/09/2027,01/13/2020,M-17-0006-CR-B,McMullen,64110001,SMUGGLING OF PERSONS,01/31/2020,06/04/2016,8.0,Denied on 06/24/2020,06/2021,NOT IN REVIEW PROCESS,100,Human_Trafficking,30_to_40,783
